# 4) Functions Challenge — Capstone Exercises

**Goals:** design small, reusable functions; pass functions around; safe defaults; error handling; docstrings & tests; gluing pieces into a mini pipeline.

You can solve these by composing helpers you wrote earlier.

### A) Mini ETL: transactions → daily totals

Input: CSV lines like:

```
2025-08-01,alice,+10.50
2025-08-01,alice,-2
2025-08-02,bob,+5
bad,line
```

Produce: `{"2025-08-01": {"alice": 8.5}, "2025-08-02": {"bob": 5.0}}`

```python
def parse_line(line):
    """
    Return (date, user, amount_float) or None if invalid.
    Amount may have '+' or '-' sign; spaces allowed.
    """
    ...
def accumulate(lines):
    """
    Use parse_line; return nested dict date->{user: total}.
    """
    ...
lines = ["2025-08-01,alice,+10.50","2025-08-01,alice,-2","2025-08-02,bob,+5","bad,line"]
out = accumulate(lines)
assert out == {"2025-08-01":{"alice":8.5}, "2025-08-02":{"bob":5.0}}
```

In [1]:
def parse_line(line):
    """
    Return (date, user, amount_float) or None if invalid.
    Amount may have '+' or '-' sign; spaces allowed.
    """
    try:
        date, user, amount = line.split(",")
        amount_float = float(amount)
        return date, user, amount_float
    except ValueError:
        return None
    
def accumulate(lines):
    """
    Use parse_line; return nested dict date->{user: total}.
    """
    result = {}
    for line in lines:
        parsed = parse_line(line)
        if parsed is None:
            continue
        date, user, amount = parsed
        if date not in result:
            result[date] = {}
        if user not in result[date]:
            result[date][user] = 0.0
        result[date][user] += amount
    return result

lines = ["2025-08-01,alice,+10.50","2025-08-01,alice,-2","2025-08-02,bob,+5","bad,line"]
out = accumulate(lines)
assert out == {"2025-08-01":{"alice":8.5}, "2025-08-02":{"bob":5.0}}



### B) Validation + normalization + mapping

```python
def normalize_user(name, email):
    """
    Return dict {'name': 'Title Case', 'email': lower} or raise ValueError.
    """
    ...
def map_users(rows, *, validator=normalize_user):
    """
    rows: list of (name,email). Use validator to normalize each; skip invalid.
    """
    ...
rows = [(" ada lovelace ","ADA@EXAMPLE.COM"), ("bad","noats")]
ok = map_users(rows)
assert ok == [{"name":"Ada Lovelace","email":"ada@example.com"}]
```


In [2]:
def normalize_user(name, email):
    """
    Return dict {'name': 'Title Case', 'email': lower} or raise ValueError.
    """
    name = name.strip().title()
    email = email.strip().lower()
    if "@" not in email or "." not in email.split("@")[-1]:
        raise ValueError("Invalid email")
    return {"name": name, "email": email}

def map_users(rows, *, validator=normalize_user):
    """
    rows: list of (name,email). Use validator to normalize each; skip invalid.
    """
    result = []
    for name, email in rows:
        try:
            user = validator(name, email)
            result.append(user)
        except ValueError:
            continue
    return result

rows = [(" ada lovelace ","ADA@EXAMPLE.COM"), ("bad","noats")]
ok = map_users(rows)
assert ok == [{"name":"Ada Lovelace","email":"ada@example.com"}]